# 앙상블 검색기(Ensemble Retriever)

`EnsembleRetriever`는 여러 검색기를 결합하여 더 강력한 검색 결과를 제공하는 LangChain의 기능입니다. 이 검색기는 다양한 검색 알고리즘의 장점을 활용하여 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

**주요 특징**
1. 여러 검색기 통합: 다양한 유형의 검색기를 입력으로 받아 결과를 결합합니다.
2. 결과 재순위화: [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) 알고리즘을 사용하여 결과의 순위를 조정합니다.
3. 하이브리드 검색: 주로 `sparse retriever`(예: BM25)와 `dense retriever`(예: 임베딩 유사도)를 결합하여 사용합니다.

**장점**
- Sparse retriever: 키워드 기반 검색에 효과적
- Dense retriever: 의미적 유사성 기반 검색에 효과적

이러한 상호 보완적인 특성으로 인해 `EnsembleRetriever`는 다양한 검색 시나리오에서 향상된 성능을 제공할 수 있습니다.

자세한 내용은 [LangChain 공식 문서](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)를 참조하세요.


In [7]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [8]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("korean_ensemble_retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
korean_ensemble_retriever


## 문서 로드 및 전처리

In [35]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/MD_menu_price.txt')
docs = loader.load()

In [36]:
docs = docs[0].page_content

In [37]:
def nums(text):
    return text.replace(u'.0', u'')
def clean_text(text):
    import re
    return re.sub(r'[^가-힣a-zA-Z0-9\s]', '', str(text))
def clean_xao(text):
    return text.replace(u'\xa0', u' ')

In [38]:
docs = nums(docs)
docs = clean_text(docs)
docs = clean_xao(docs)

In [39]:
def save_text_to_file(text, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

save_text_to_file(docs, '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/processed_docs.txt')
print("텍스트가 'processed_docs.txt' 파일에 저장되었습니다.")

텍스트가 'processed_docs.txt' 파일에 저장되었습니다.


## 본 코드

- `EnsembleRetriever`를 초기화하여 `BM25Retriever`와 `FAISS` 검색기를 결합합니다. 각 검색기의 가중치를 설정됩니다.


In [1]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/processed_docs.txt')
docs = loader.load()

In [42]:
docs

[Document(metadata={'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/processed_docs.txt'}, page_content='메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다\n\n메뉴이름  베토디 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함 이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 251g이다 열량은 621kcal이다 단품가격은 7600원 세트가격은 9500원이다\n\n메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로 입맛도 기분도 화끈하게 이다 식재료는 난류대두밀토마토닭고기가 들어간다 중량은 246g이다 열량은 501kcal이다 단품가격은 6300원 세트가격은 8400원이다\n\n메뉴이름  토마토 치즈 비프 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 신선한 토마토와 고소한 치즈버거의 조화 이다 식재료는 난류우유대두밀토마토쇠고기가 들어간다 중량은 200g이다 열량은 403kcal이다 단품가격은 4600원 세트가격은 6900원이다\n\n메뉴이름  더블 쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 좋아하는건 더 많이 즐기시라고 두 배 이상 커진  파운드 비프 패티가 두 장 그램육즙이 풍부한 고기 맛을 그대로 살린  순 쇠고기 패티 두 장과 치즈 두 장이 입안 가득 완벽하게 조화되는 놀라운 맛 이다 식재료는 우유

In [2]:
from langchain.text_splitter import CharacterTextSplitter

# CharacterTextSplitter를 사용하여 문서 분할
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 173, which is longer than the specified 100
Created a chunk of size 163, which is longer than the specified 100
Created a chunk of size 212, which is longer than the specified 100
Created a chunk of size 143, which is longer than the specified 100
Created a chunk of size 236, which is longer than the specified 100
Created a chunk of size 242, which is longer than the specified 100
Created a chunk of size 208, which is longer than the specified 100
Created a chunk of size 198, which is longer than the specified 100
Created a chunk of size 171, which is longer than the specified 100
Created a chunk of size 201, which is longer than the specified 100
Created a chunk of size 187, which is longer than the specified 100
Created a chunk of size 173, which is longer than the specified 100
Created a chunk of size 182, which is longer than the specified 100
Created a chunk of size 153, which is longer than the specified 100
Created a chunk of size 175, which is longer tha

In [44]:
print("분할된 문서 수:", len(split_docs))

분할된 문서 수: 27


In [4]:
split_docs

[Document(metadata={'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/processed_docs.txt'}, page_content='메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다'),
 Document(metadata={'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/processed_docs.txt'}, page_content='메뉴이름  베토디 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함 이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 251g이다 열량은 621kcal이다 단품가격은 7600원 세트가격은 9500원이다'),
 Document(metadata={'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/processed_docs.txt'}, page_content='메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로 입맛도 기분도 화끈하게 이다 식재료는 난류대두밀토마토닭고기가 들어간다 중량은 246g이다 열량은 501kcal이다 단품가격은 6300원 세트가격은 8400원이다'),
 Document(metadata={'source': '/home/yoojin/ML/aiffel/HackaThon/modu

In [3]:
from kiwipiepy import Kiwi
from konlpy.tag import Kkma, Okt
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

kkma = Kkma()
okt = Okt()
kiwi = Kiwi()

In [47]:
# Kiwi tokenizer를 사용하여 토큰화, 엄청 빠름(0.0초)
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

# 결과 확인
print("\n첫 번째 분할 문서의 토큰화 결과:")
tokenized_result = kiwi_tokenize(split_docs[0].page_content)
print(tokenized_result)
print(len(tokenized_result))


첫 번째 분할 문서의 토큰화 결과:
['메뉴', '이름', '맥크리스피', '스리라차', '마요', '버거', '카테고리', '는', '버거', '이', '며', '신', '메뉴', '이', '다', '이', '메뉴', '는', '빠삭', '하', 'ᆫ', '통닭', '다리', '살', '케이준', '패티', '에', '스리라차', '말', '어요', '소스', '를', '더', '하', '어', '매콤', '고소', '하', 'ᆫ', '맛', '이', '다', '식재료', '는', '난류', '우유', '대두', '밀', '토마토', '닭고기', '쇠고기', '가', '들어가', 'ᆫ다', '중량', '은', '289', 'g', '이', '다', '열량', '은', '663', 'kcal', '이', '다', '단', '품', '가격', '은', '8600', '원', '세트', '가격', '은', '10500', '원', '이', '다']
79


In [48]:
# kkma를 사용하여 토큰화, 시간이 꽤 걸림(5.8초)
kkma_tokenized_docs = []
for doc in split_docs:
    tokens = kkma.morphs(doc.page_content)
    kkma_tokenized_docs.append(tokens)


print("\n첫 번째 분할 문서의 토큰화 결과:")
print(kkma_tokenized_docs[0])  # 첫 분할 문서 토큰화 결과
len(tokens)


첫 번째 분할 문서의 토큰화 결과:
['메뉴', '이름', '맥', '크리스피', '스', '리라', '차', '마', '요', '버거', '카테고리', '는', '버거', '이', '며', '신', '메뉴', '이', '다', '이', '메뉴', '는', '빠삭', '하', 'ㄴ', '통닭', '다리', '살', '케이', '주', 'ㄴ', '패티', '에', '스', '리라', '차', '마', '요', '소스', '를', '더하', '어', '매콤', '고소한', '맛', '이', '다', '식재료', '는', '난류', '우유', '대두', '밀', '토마토', '닭고기', '쇠고기', '가', '들어가', 'ㄴ다', '중량', '은', '289', 'g', '이다', '열량', '은', '663', 'kcal', '이', '다', '단', '품', '가격', '은', '8600', '원', '세트', '가격', '은', '10500', '원', '이', '다']


64

In [49]:
# okt를 사용하여 토큰화, 2번째로 빠름(0.2초)
okt_tokenized_docs = []
for doc in split_docs:
    tokens = okt.morphs(doc.page_content)
    okt_tokenized_docs.append(tokens)


print("\n첫 번째 분할 문서의 토큰화 결과:")
print(okt_tokenized_docs[0])  # 첫 분할 문서 토큰화 결과
len(tokens)


첫 번째 분할 문서의 토큰화 결과:
['메뉴', '이름', '맥', '크리스', '피', '스리', '라차', '마', '요', '버거', '카테고리', '는', '버거', '이며', '신', '메뉴', '이다', '이', '메뉴', '는', '빠삭', '한', '통닭', '다리', '살', '케이준', '패티', '에', '스리', '라차', '마', '요', '소스', '를', '더해', '매콤', '고소한', '맛', '이다', '식', '재료', '는', '난류', '우유', '대', '두', '밀', '토마토', '닭고기', '쇠고기', '가', '들어간다', '중량', '은', '289', 'g', '이다', '열량', '은', '663', 'kcal', '이다', '단', '품', '가격', '은', '8600원', '세트', '가격', '은', '10500원', '이다']


50

In [5]:
def kkma_tokenize(text):
    return [token for token in kkma.morphs(text)]
def okt_tokenize(text):
    return [token for token in okt.morphs(text)]
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [6]:
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large")  # OpenAI 임베딩을 사용합니다.
store = LocalFileStore("./.cache2")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,
    document_embedding_cache=store,
    namespace=embedding.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

faiss_vectorstore = FAISS.from_documents(
    split_docs,
    cached_embedder,
)
faiss = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

In [ ]:
# caching할 수 있으려나..? 해서 일단 보관
# from functools import lru_cache

# @lru_cache(maxsize=None)
# def cached_kiwi_tokenize(text):
#     return tuple(kiwi_tokenize(text))

# @lru_cache(maxsize=None)
# def cached_kkma_tokenize(text):
#     return tuple(kkma_tokenize(text))

# @lru_cache(maxsize=None)
# def cached_okt_tokenize(text):
#     return tuple(okt_tokenize(text))

# kiwi_bm25 = BM25Retriever.from_documents(split_docs, preprocess_func=cached_kiwi_tokenize)
# kkma_bm25 = BM25Retriever.from_documents(split_docs, preprocess_func=cached_kkma_tokenize)
# okt_bm25 = BM25Retriever.from_documents(split_docs, preprocess_func=cached_okt_tokenize)

In [9]:
kiwi_bm25 = BM25Retriever.from_documents(split_docs, preprocess_func=kiwi_tokenize)
kkma_bm25 = BM25Retriever.from_documents(split_docs, preprocess_func=kkma_tokenize)
okt_bm25 = BM25Retriever.from_documents(split_docs, preprocess_func=okt_tokenize)

kiwi_bm25.k = 4
kkma_bm25.k = 4
okt_bm25.k = 4

In [14]:
kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
kkma_bm25_faiss_73 = EnsembleRetriever(
    retrievers=[kkma_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
kkma_bm25_faiss_37 = EnsembleRetriever(
    retrievers=[kkma_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
okt_bm25_faiss_73 = EnsembleRetriever(
    retrievers=[okt_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
okt_bm25_faiss_37 = EnsembleRetriever(
    retrievers=[okt_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
kiwibm25_faiss_55 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.5, 0.5],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
kkma_bm25_faiss_55 = EnsembleRetriever(
    retrievers=[kkma_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.5, 0.5],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
okt_bm25_faiss_55 = EnsembleRetriever(
    retrievers=[okt_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.5, 0.5],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

In [15]:
def results(query):
    print(f"kiwibm25_faiss_73:{kiwibm25_faiss_73.invoke(query)[0].page_content}")
    print("=================================================")
    print(f"kiwi_bm25_faiss_55: {kiwibm25_faiss_55.invoke(query)[0].page_content}")
    print("=================================================")
    print(f"kiwibm25_faiss_37:{kiwibm25_faiss_37.invoke(query)[0].page_content}")
    print("=================================================")
    print(f"kkma_bm25_faiss_73:{kkma_bm25_faiss_73.invoke(query)[0].page_content}")
    print("=================================================")
    print(f"kkma_bm25_faiss_55: {kkma_bm25_faiss_55.invoke(query)[0].page_content}")
    print("=================================================")
    print(f"kkma_bm25_faiss_37:{kkma_bm25_faiss_37.invoke(query)[0].page_content}")
    print("=================================================")
    print(f"okt_bm25_faiss_73:{okt_bm25_faiss_73.invoke(query)[0].page_content}")
    print("=================================================")
    print(f"okt_bm25_faiss_55: {okt_bm25_faiss_55.invoke(query)[0].page_content}")
    print("=================================================")
    print(f"okt_bm25_faiss_37:{okt_bm25_faiss_37.invoke(query)[0].page_content}")

## 테스트

In [16]:
# 검색 결과 문서를 가져옵니다.
query = "닭고기가 들어간 버거를 추천해줘"
results(query)

kiwibm25_faiss_73:메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로 입맛도 기분도 화끈하게 이다 식재료는 난류대두밀토마토닭고기가 들어간다 중량은 246g이다 열량은 501kcal이다 단품가격은 6300원 세트가격은 8400원이다
kiwi_bm25_faiss_55: 메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로 입맛도 기분도 화끈하게 이다 식재료는 난류대두밀토마토닭고기가 들어간다 중량은 246g이다 열량은 501kcal이다 단품가격은 6300원 세트가격은 8400원이다
kiwibm25_faiss_37:메뉴이름  더블 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 진한 불고기 소스의 패티가 두 장 여기에 고소한 치즈와 마요네즈 신선한 양상추를 곁들인 깊고 풍부한 맛   이다 식재료는 난류우유대두밀돼지고기조개굴가 들어간다 중량은 237g이다 열량은 635kcal이다 단품가격은 5300원 세트가격은 7500원이다
kkma_bm25_faiss_73:메뉴이름  맥크리스피 디럭스 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 촉촉한 포테이토 브리오쉬 번 스페셜 스모키 소스가 선사하는 놀랍도록 새로운 맛의 치킨 버거  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 276g이다 열량은 614kcal이다 단품가격은 7600원 세트가격은 9200원이다
kkma_bm25_faiss_55: 메뉴이름  맥크리스피 디럭스 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 촉촉한 포테이토 브리오쉬 번 스페셜 스모키 소스가 선사하는

In [18]:
query = "닭고기가 들어가지 않은 버거를 추천해줘"
results(query)

kiwibm25_faiss_73:메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로 입맛도 기분도 화끈하게 이다 식재료는 난류대두밀토마토닭고기가 들어간다 중량은 246g이다 열량은 501kcal이다 단품가격은 6300원 세트가격은 8400원이다
kiwi_bm25_faiss_55: 메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로 입맛도 기분도 화끈하게 이다 식재료는 난류대두밀토마토닭고기가 들어간다 중량은 246g이다 열량은 501kcal이다 단품가격은 6300원 세트가격은 8400원이다
kiwibm25_faiss_37:메뉴이름  더블 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 진한 불고기 소스의 패티가 두 장 여기에 고소한 치즈와 마요네즈 신선한 양상추를 곁들인 깊고 풍부한 맛   이다 식재료는 난류우유대두밀돼지고기조개굴가 들어간다 중량은 237g이다 열량은 635kcal이다 단품가격은 5300원 세트가격은 7500원이다
kkma_bm25_faiss_73:메뉴이름  맥크리스피 디럭스 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 촉촉한 포테이토 브리오쉬 번 스페셜 스모키 소스가 선사하는 놀랍도록 새로운 맛의 치킨 버거  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 276g이다 열량은 614kcal이다 단품가격은 7600원 세트가격은 9200원이다
kkma_bm25_faiss_55: 메뉴이름  맥크리스피 디럭스 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는  통닭다리살 겉바속촉 케이준 치킨 패티 촉촉한 포테이토 브리오쉬 번 스페셜 스모키 소스가 선사하는

In [19]:
query = "신메뉴는 뭐가있어?"
results(query)

kiwibm25_faiss_73:메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kiwi_bm25_faiss_55: 메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kiwibm25_faiss_37:메뉴이름  탄산 미디움 카테고리는 음료 이며 신메뉴가 아니다 이 메뉴는 갈증해소 뿐만이 아니라 기분까지 상쾌하게 코카콜라 이다 단품가격은 2600원이다
kkma_bm25_faiss_73:메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kkma_bm25_faiss_55: 메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 

In [20]:
query = "매콤한 메뉴는 뭐가있어?"
results(query)

kiwibm25_faiss_73:메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다
kiwi_bm25_faiss_55: 메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다
kiwibm25_faiss_37:메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다
kkma_bm25_faiss_73:메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kkma_bm25_faiss_55: 메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사

In [21]:
query = "매콤하지 않은 메뉴는 뭐가있어?"
results(query)

kiwibm25_faiss_73:메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로 입맛도 기분도 화끈하게 이다 식재료는 난류대두밀토마토닭고기가 들어간다 중량은 246g이다 열량은 501kcal이다 단품가격은 6300원 세트가격은 8400원이다
kiwi_bm25_faiss_55: 메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로 입맛도 기분도 화끈하게 이다 식재료는 난류대두밀토마토닭고기가 들어간다 중량은 246g이다 열량은 501kcal이다 단품가격은 6300원 세트가격은 8400원이다
kiwibm25_faiss_37:메뉴이름  맥크리스피 스리라차 마요 버거 카테고리는 버거 이며 신메뉴이다 이 메뉴는 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛  이다 식재료는 난류우유대두밀토마토닭고기쇠고기가 들어간다 중량은 289g이다 열량은 663kcal이다 단품가격은 8600원 세트가격은 10500원이다
kkma_bm25_faiss_73:메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로 입맛도 기분도 화끈하게 이다 식재료는 난류대두밀토마토닭고기가 들어간다 중량은 246g이다 열량은 501kcal이다 단품가격은 6300원 세트가격은 8400원이다
kkma_bm25_faiss_55: 메뉴이름  맥스파이시 상하이 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지

In [22]:
query = "가장 열량이 낮은 메뉴는 뭐가있어?"
results(query)

kiwibm25_faiss_73:메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kiwi_bm25_faiss_55: 메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kiwibm25_faiss_37:메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kkma_bm25_faiss_73:메뉴이름  에그 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다  이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 222g이다 열량은 491kcal이다 단품가격은 5100원 세트가격은 7300원이다
kkma_bm25_faiss_55: 메뉴이름  에그 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다  이다 식재료는 난류대두밀

In [23]:
query = "가장 양이 적은 버거는 뭐야?"
results(query)

kiwibm25_faiss_73:메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kiwi_bm25_faiss_55: 메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kiwibm25_faiss_37:메뉴이름  1955 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 g 두툼한 순 쇠고기 패티 특별한  소스에 깊은 풍미의 그릴드 어니언까지 맥도날드가 처음 생긴 년의 맛을 담은 영원한 오리지널  버거    이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 264g이다 열량은 572kcal이다 단품가격은 7200원 세트가격은 9100원이다
kkma_bm25_faiss_73:메뉴이름  빅맥 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는  순 쇠고기 패티 두 장에 빅맥만의 특별한 소스 입안에서 살살 녹는 치즈와 신선한 양상추 양파 그리고  피클까지 년 넘게 전 세계인의 입맛을 사로 잡은 버거의 대명사  이다 식재료는 난류우유대두밀쇠고기가 들어간다 중량은 223g이다 열량은 582kcal이다 단품가격은 6300원 세트가격은 8500원이다
kkma_bm25_faiss_55: 메뉴이름  빅맥

In [24]:
query = "해산물이 들어간 버거는 뭐가있어?"
results(query)

kiwibm25_faiss_73:메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kiwi_bm25_faiss_55: 메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kiwibm25_faiss_37:메뉴이름  에그 불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다  이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 222g이다 열량은 491kcal이다 단품가격은 5100원 세트가격은 7300원이다
kkma_bm25_faiss_73:메뉴이름  쿼터파운더 치즈 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 쿼터파운더라는 이름에서 알 수 있듯이 두 배 이상 커진 파운드그램 비프와 부드러운 치즈 두 장의 환상궁합두툼한 순 쇠고기 패티와 신선한 치즈의 풍부한 맛으로 세계적으로 사랑받고 있는 맥도날드의 대표적인 프리미엄 버거 이다 식재료는 우유대두밀토마토쇠고기가 들어간다 중량은 198g이다 열량은 536kcal이다 단품가격은 6300원 세트가격은 8900원이다
kkma_bm25_faiss_55: 메뉴이름  쿼터파운더 치즈 버거 카테고리

In [29]:
query = "모든 메뉴 통틀어서 제일 싼 메뉴는 뭐야?"
results(query)

kiwibm25_faiss_73:메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kiwi_bm25_faiss_55: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kiwibm25_faiss_37:메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kkma_bm25_faiss_73:메뉴이름  후렌치후라이 스몰 카테고리는 사이드이며 신메뉴가 아니다 이 메뉴는 통으로 썰어낸 감자를 맥도날드만의 노하우로 튀겨낸 남다른 맛과 바삭함 맥도날드의 역사가 담긴 월드 클래스 후렌치 후라이  이다 단품가격은 2300원이다
kkma_bm25_faiss_55: 메뉴이름  후렌치후라이 스몰 카테고리는 사이드이며 신메뉴가 아니다 이 메뉴는 통으로 썰어낸 감자를 맥도날드만의 노하우로 튀겨낸 남다른 맛과 바삭함 맥도날드의 역사가 담긴 월드 클래스 후렌치 후라이  이다 단품가격은 2300원이다
kkma_bm25_faiss_37:메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식

In [27]:
query = "제일 싼 버거는 뭐야?"
results(query)

kiwibm25_faiss_73:메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kiwi_bm25_faiss_55: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kiwibm25_faiss_37:메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kkma_bm25_faiss_73:메뉴이름  1955 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 g 두툼한 순 쇠고기 패티 특별한  소스에 깊은 풍미의 그릴드 어니언까지 맥도날드가 처음 생긴 년의 맛을 담은 영원한 오리지널  버거    이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 264g이다 열량은 572kcal이다 단품가격은 7200원 세트가격은 9100원이다
kkma_bm25_faiss_55: 메뉴이름  1955 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 g 두툼한 순 쇠고기 패티 특별한  소스에 깊은 풍미의 그릴드 어니언까지 맥도날드가 처음 생긴 년의 맛을 담은 영원한 오리지널  버거    이다 식재료는 난류우유대두밀돼지고기토마토쇠고기가 들어간다 중량은 264g이다 열량은 572kcal이다

In [28]:
query = "세트가격이 제일 싼 버거는 뭐야?"
results(query)

kiwibm25_faiss_73:메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kiwi_bm25_faiss_55: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kiwibm25_faiss_37:메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kkma_bm25_faiss_73:메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kkma_bm25_faiss_55: 메뉴이름  불고기 버거 카테고리는 버거 이며 신메뉴가 아니다 이 메뉴는 한국인의 입맛에 딱 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈 신선한 양상추의 맛있는 조합   이다 식재료는 난류대두밀돼지고기조개굴가 들어간다 중량은 167g이다 열량은 409kcal이다 단품가격은 4200원 세트가격은 6600원이다
kkma_bm25_faiss_37:메뉴이름  